In [169]:
import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import re
from scipy import stats
import seaborn as sns
import scipy.stats as sp

In [139]:
from google.cloud import bigquery

# Set billing
client = bigquery.Client(project = 'etsy-bigquery-adhoc-prod')

# sql = """
# (with prolist as 
#     (SELECT visit_id, sum(cost) / 100 as spend 
#     FROM `etsy-data-warehouse-prod.rollups.prolist_click_visits` 
#     WHERE _date >= '2023-01-01'
#     group by 1),
# visits_receipts as 
#     (select distinct visit_id, receipt_id
#     from etsy-data-warehouse-prod.transaction_mart.transactions_visits
#     where date >= '2023-01-01'),
# tot_rev as 
#     (select visit_id, sum(attr_rev) as rev, sum(etsy_ads_revenue) as osa_rev, sum(ltv_rev) as ltv_rev,
#     sum(attr_rev - ltv_rev - etsy_ads_revenue) as comm_rev,
#     from visits_receipts 
#     left join etsy-data-warehouse-prod.buyatt_mart.buyatt_analytics_clv using (receipt_id)
#     group by 1),
# bucketid_level  as
#     (select  a.config_flag_param as experiment_id,a.variant_id, 
#     bucketing_id,
#     count(distinct v.visit_id) as visits,
#     sum( coalesce(spend,0) )  as prolist_rev,
#     sum( coalesce(ltv_rev,0) ) as ltv_rev,
#     sum( coalesce(osa_rev,0) ) as osa_rev,
#     sum( coalesce(comm_rev,0) ) as comm_rev,
#     sum( coalesce(spend,0) + coalesce(rev,0) ) as total_rev,
#     max(distinct case when converted=1 then 1 else 0 end) as is_converted,
#     from `etsy-data-warehouse-dev.tnormil.lex_visit_level_exp` a
#     join etsy-data-warehouse-prod.weblog.visits v on a.bucketing_id = v.browser_id
#     left join prolist p on p.visit_id = v.visit_id
#     left join tot_rev r on p.visit_id = r.visit_id
#     where v._date >= '2023-01-01'
#     -- and a.bucketing_ts >= v.start_datetime 
#     and timestamp(format_timestamp("%Y-%m-%d %H:%M:%M %Z",  start_datetime )) >= timestamp(format_timestamp("%Y-%m-%d %H:%M:%M %Z", bucketing_ts))
#     and date(v.start_datetime) <= end_date
#     and date(v.start_datetime) >= start_date
#     group by 1,2,3)
# select experiment_id, variant_id, 
# count(distinct bucketing_id) as bucketing_id, 
# sum(visits) as visits,
# safe_divide( sum(is_converted), count(distinct bucketing_id) ) as cvr,
# avg(prolist_rev) as mean_prolist_rev,
# avg(ltv_rev) as mean_ltv_rev,
# avg(osa_rev) as mean_osa_rev,
# avg(comm_rev) as mean_comm_rev,
# avg(total_rev) as mean_total_rev,
# stddev_samp(prolist_rev) as std_prolist_rev,
# stddev_samp(ltv_rev) as std_ltv_rev,
# stddev_samp(osa_rev) as std_osa_rev,
# stddev_samp(comm_rev) as std_comm_rev,
# stddev_samp(total_rev) as std_total_rev
# from bucketid_level
# group by 1,2)
# """

sql = """
select a.experiment_id, 
a.variant_id,
a.start_date,
a.end_date,
bucketing_id,
a.visits,
cvr,
mean_prolist_rev,
mean_cvr_prolist_rev,
mean_ltv_rev,
mean_osa_rev,
mean_comm_rev,
mean_total_rev,
mean_total_rev_ex_ncp,
-- safe_divide(mean_ltv_rev*bucketing_id,bucketing_id*cvr) as mean_ltv_rev_per_converting, 
-- safe_divide(mean_total_rev*bucketing_id,bucketing_id*cvr) as mean_total_rev_per_converting, 
std_prolist_rev,
std_cvr_prolist_rev,
std_ltv_rev,
std_osa_rev,
std_comm_rev,
std_total_rev,
std_total_rev_ex_ncp,
safe_divide(a.visits, b.visits) - 1 as visits_diff,
safe_divide(a.cvr, b.conversion_rate) - 1 as cvr_diff,
safe_divide(a.mean_prolist_rev, b.prolist_total_spend_per_browser/100) - 1 as prolist_total_spend_per_browser_diff,
safe_divide(a.mean_osa_rev*bucketing_id, b.offsite_ads_revenue_sum/100) - 1 as offsite_ads_revenue_sum_diff,
from `etsy-data-warehouse-dev.tnormil.lex_dev_catapult_stat` a
left join `etsy-data-warehouse-dev.tnormil.lex_prod_catapult_stat` b using (experiment_id, variant_id)
"""

query_job = client.query(sql) # Run query
results = query_job.result()
df = results.to_dataframe() # Save results to dataframe

In [140]:
df.head()

,experiment_id,variant_id,start_date,end_date,bucketing_id,visits,cvr,mean_prolist_rev,mean_cvr_prolist_rev,mean_ltv_rev,...,std_cvr_prolist_rev,std_ltv_rev,std_osa_rev,std_comm_rev,std_total_rev,std_total_rev_ex_ncp,visits_diff,cvr_diff,prolist_total_spend_per_browser_diff,offsite_ads_revenue_sum_diff
0,cnc.desktop_lp_add_shop_image,off,2024-01-26,2024-02-06,8000699,17191742,0.05907,0.104945204,0.042197101,0.153152135,...,0.35639,1.87395,1.32275,4.39312,6.12959,6.12071,0.00765,0.10568,0.07817,0.23524
1,cnc.desktop_lp_add_shop_image,on,2024-01-26,2024-02-06,8012169,17183789,0.05897,0.104567555,0.042155284,0.151406165,...,0.35597,1.83424,1.29949,4.28349,5.97553,5.96639,0.00766,0.10550,0.07868,0.23032
2,cnc.mweb_lp_add_shop_image,on,2024-01-26,2024-02-01,14566121,23682900,0.01737,0.052321585,0.012752241,0.055854918,...,0.17850,0.93738,0.74855,1.32140,2.51577,2.50532,0.00694,0.01606,0.06425,0.09948
3,cnc.mweb_lp_add_shop_image,off,2024-01-26,2024-02-01,14566370,23675610,0.01746,0.052461539,0.012819644,0.055894604,...,0.17719,0.90544,0.73862,1.38529,2.52252,2.51238,0.00701,0.01596,0.06419,0.09153
4,cnc.free_shipping_badge_ext_lp,on,2024-02-01,2024-02-15,5716911,12177660,0.02269,0.100466101,0.026160547,0.087406687,...,0.32592,1.38040,1.18021,2.68637,4.34897,4.33016,0.01742,0.01880,0.07410,0.11623


In [192]:
def power_proportion_metric(
    mean_control: np.ndarray,
    sample_size_control: np.ndarray,
    sample_size_treatment: np.ndarray,
    percent_lift: np.ndarray,
    alpha: float = 0.05,
) -> np.ndarray:
    """Calculates the current power of the proportion metric (vectorized).

    Taken from Etsyweb:
    'Power calculation for two sample test for proportions taken directly
    from the R function `pwr.2p2n.test`
    https://www.rdocumentation.org/packages/pwr/versions/1.2-0/topics/pwr.2p2n.test'

    Args:
        mean_control: The mean of the control group.
        sample_size_control: The sample size of the control group.
        sample_size_treatment: The sample size of the treatment group.
        percent_lift: The expected percent lift in the metric (scale: [0, 100]).
        alpha: The significance threshold.

    Returns:
        A numpy array containing the power calculations for the proportion
        metrics provided — or NaN if these calculations can't be computed.
    """
    p1: np.ndarray = mean_control
    p2: np.ndarray = mean_control * (1 + percent_lift / 100)
    p2 = np.minimum(p2, 1)

    result: np.ndarray = np.full(len(mean_control), np.nan)
    row_idx: np.ndarray = (
        (p1 != 0) & (p2 != 0) & (sample_size_control + sample_size_treatment > 0)
    )

    # Non-directional h, magnitude of difference
    h: np.ndarray = np.abs(
        2 * np.arcsin(np.sqrt(np.maximum(p1[row_idx], 0)))
        - 2 * np.arcsin(np.sqrt(np.maximum(p2[row_idx], 0)))
    )

    temp: np.ndarray = np.maximum(
        (sample_size_control[row_idx] * sample_size_treatment[row_idx])
        / (sample_size_control[row_idx] + sample_size_treatment[row_idx]),
        0,
    )
    power: np.ndarray = (
        1
        - sp.norm.cdf(-sp.norm.ppf(alpha / 2) - h * np.sqrt(temp))
        + sp.norm.cdf(sp.norm.ppf(alpha / 2) - h * np.sqrt(temp))
    )
    result[row_idx] = power

    return result

In [193]:
def power_mean_metric(
    mean_control: np.ndarray,
    mean_treatment: np.ndarray,
    var_control: np.ndarray,
    var_treatment: np.ndarray,
    sample_size_control: np.ndarray,
    sample_size_treatment: np.ndarray,
    percent_lift: np.ndarray,
    is_log_metric: np.ndarray = None,
    #alpha: float = DEFAULT_ALPHA,
    alpha: float = 0.05,
) -> np.ndarray:
    """Calculates the current power of the mean metric (vectorized).

    Args:
        mean_control: The mean of the control group.
        mean_treatment: The mean of the treatment group.
        var_control: The variance of the individual observations in the control
            group.
        var_treatment: The variance of the individual observations in the
            treatment group.
        sample_size_control: The sample size of the control group.
        sample_size_treatment: The sample size of the treatment group.
        percent_lift: The expected percent lift in the metric (scale: [0, 100]).
        is_log_metric: Boolean indicating whether the row pertains to a log
            metric or not.
        alpha: The significance threshold.

    Returns:
        A numpy array containing the power calculations for the mean metrics
        provided — or NaN if these calculations can't be computed.
    """
    if is_log_metric is None:
        # If not supplied, assume all metrics are NOT log metrics
        is_log_metric = np.full(len(mean_control), False)

    diff_to_observe: np.ndarray = mean_control * (percent_lift / 100)

    std_error: np.ndarray = np.full(len(mean_control), np.nan)
    # Guard against divide-by-0 errors
    row_idx: np.ndarray = (
        (sample_size_control > 0)
        & (sample_size_treatment > 0)
        &
        # According to Etsyweb, there is no power calculation for log metrics.
        ~is_log_metric
    )
    std_error[row_idx] = np.sqrt(
        (var_control[row_idx] / sample_size_control[row_idx])
        + (var_treatment[row_idx] / sample_size_treatment[row_idx])
    )

    result: np.ndarray = np.full(len(mean_treatment), np.nan)
    row_idx = std_error > 0
    result[row_idx] = sp.norm.cdf(
        (diff_to_observe[row_idx] / std_error[row_idx]) + sp.norm.ppf(alpha / 2)
    )

    return result

In [202]:
from scipy.stats import binomtest
import numpy as np
from scipy.stats import ttest_ind_from_stats
from statsmodels.stats.proportion import test_proportions_2indep
from statsmodels.stats.proportion import power_proportions_2indep
from statsmodels.stats.power import TTestIndPower 
import math 
import statistics 

results = df.loc[df.variant_id != 'off' ,['experiment_id','variant_id']].copy()

experiments = df['experiment_id'].drop_duplicates().tolist()

measures =  df.loc[:,'cvr':].columns.tolist()

measures = [m for m in measures if 'std_' not in m]

for m in measures:
    
    results[m.replace("mean_", "")+'_treatment'] = np.nan
    results[m.replace("mean_", "")+'_control'] = np.nan
    results[m.replace("mean_", "")+'_lift'] = np.nan
    results[m.replace("mean_", "")+'_sig'] = np.nan
    results[m.replace("mean_", "")+'_pwr'] = np.nan

def get_num_variants(experiment_id):
    
    variants = df.loc[ (df.experiment_id == e), 'variant_id' ].drop_duplicates().tolist()
    
    return len(variants)

def alpha_correction(experiment_id):   
    variants = get_num_variants(experiment_id)
    if variants > 2:
        # number_of_comparisons = variants * (variants-1) / 2
        return 0.10 / (variants-1)
    else:
        return 0.10


def get_sig_threshold(experiment_id, variant_id):
    
    for m in measures:
        
        treatment_m = float(df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m ].values.item())
        control_m = float(df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , m ].values.item())
        
        treatment_pop = df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item()
        control_pop =  df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , 'bucketing_id' ].item()
        
        lift =  treatment_m/control_m - 1
        
        results.loc[ (results.experiment_id == experiment_id)&(results.variant_id == variant_id) , m.replace("mean_", "")+'_lift' ] = lift
        results.loc[ (results.experiment_id == experiment_id)&(results.variant_id == variant_id) , m.replace("mean_", "")+'_treatment' ] = treatment_m
        results.loc[ (results.experiment_id == experiment_id)&(results.variant_id == variant_id) , m.replace("mean_", "")+'_control' ] = control_m
                
        
        if "mean_" in m:
            
            treatment_std = float(df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m.replace("mean_", "std_") ].values.item())
            control_std = float(df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , m.replace("mean_", "std_") ].values.item())
            
            sig_threshold = ttest_ind_from_stats(
            
            treatment_m, 
                
            treatment_std,
            
            treatment_pop, 
            
            control_m, 
                
            control_std,
            
            control_pop,
                
            equal_var=True, alternative='two-sided')
                        
            pval = sig_threshold.pvalue
            
            # pooled_standard_deviation = math.sqrt( 
            #           ((n1 - 1)*SD1 * SD1 +
            #          (n2-1)*SD2 * SD2) / 
            #                       (n1 + n2-2)) 
            
            # pooled_standard_deviation = math.sqrt( 
            # ((control_pop - 1)*control_std * control_std +
            # (treatment_pop-1)*treatment_std* treatment_std) / 
            # (control_pop + treatment_pop-2)) 
            
            #pwr_threshold = TTestIndPower().power(effect_size = abs(treatment_m - control_m)/pooled_standard_deviation,
            #nobs1=control_pop, alpha=0.05, ratio=treatment_pop/control_pop, df=None, alternative='two-sided')
            
            # TTestIndPower.power(effect_size, nobs1, alpha, ratio=1, df=None, alternative='two-sided'
            
            # Define numpy arrays for stats methods
            mean_0 = np.array([treatment_m], dtype=float)
            mean_1 = np.array([control_m], dtype=float)
            var_0 = np.array([treatment_std**2])
            var_1 = np.array([control_std**2])
            sample_size_0 = np.array([treatment_pop], dtype=int)
            sample_size_1 = np.array([control_pop], dtype=int)
            
            # Power, default 1% lift
            pwr = power_mean_metric(
            mean_control = mean_0
            , mean_treatment = mean_1
            , var_control = var_0
            , var_treatment = var_1
            , sample_size_control = sample_size_0
            , sample_size_treatment = sample_size_1
            , percent_lift = 1
            , alpha = alpha_correction(experiment_id)
            )

            #pwr = pwr_threshold
        
        else:
            
#             sig_threshold = binomtest(round(df.loc[(df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m ].values.item()
#             * df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item()),
            
#             df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item(), 
            
#             df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , m ].item(), 
            
#             alternative='two-sided')
            
#             pval = sig_threshold.pvalue

            sig_threshold = test_proportions_2indep(round(df.loc[(df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m ].values.item() *
            df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item()), 
            
            df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item(), 
            
            round(df.loc[(df.experiment_id == experiment_id)&(df.variant_id == 'off') , m ].values.item() *
            df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , 'bucketing_id' ].item()), 
            
            df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , 'bucketing_id' ].item(), 
            value=None, method=None, compare='diff', alternative='two-sided', correction=True, return_results=True)
            
            pval = sig_threshold.pvalue
            
            diff = df.loc[(df.experiment_id == experiment_id)&(df.variant_id == 'off') , m ].values.item() - df.loc[(df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m ].values.item()
            
#             pwr_threshold = power_proportions_2indep(diff, 
                                                                  
#             df.loc[(df.experiment_id == experiment_id)&(df.variant_id == variant_id) , m ].values.item() + diff, 
            
#             df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , 'bucketing_id' ].item(), 
            
#             ratio=df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == variant_id) , 'bucketing_id' ].item()/df.loc[ (df.experiment_id == experiment_id)&(df.variant_id == 'off') , 'bucketing_id' ].item(), 
            
#             alpha=0.05, value=0, alternative='two-sided', return_results=True)
            
            # Define numpy arrays for stats methods
            proportion_0 = np.array([treatment_m], dtype=float)
            proportion_1 = np.array([control_m], dtype=float)
            #var_0 = np.array(var_0)
            #var_1 = np.array(var_1)
            sample_size_0 = np.array([treatment_pop], dtype=int)
            sample_size_1 = np.array([control_pop], dtype=int)
            
            pwr = power_proportion_metric(
            mean_control = proportion_0
            , sample_size_control = sample_size_0
            , sample_size_treatment = sample_size_1
            , percent_lift = 1
            , alpha = alpha_correction(experiment_id))
            
            # pwr = pwr_threshold.power
     
        results.loc[ (results.experiment_id == experiment_id)&(results.variant_id == variant_id) , m.replace("mean_", "")+'_sig' ] = pval
        results.loc[ (results.experiment_id == experiment_id)&(results.variant_id == variant_id) , m.replace("mean_", "")+'_pwr' ] = pwr
                         

In [203]:
for e in experiments:
    
    variants = df.loc[ (df.experiment_id == e)&(df.variant_id != 'off') , 'variant_id' ].drop_duplicates().tolist()
    
    for v in variants:
        
        get_sig_threshold(e, v)  
    

In [204]:
pd.options.display.float_format = '{0:.5f}'.format
results.head()

,experiment_id,variant_id,cvr_treatment,cvr_control,cvr_lift,cvr_sig,cvr_pwr,prolist_rev_treatment,prolist_rev_control,prolist_rev_lift,...,prolist_total_spend_per_browser_diff_treatment,prolist_total_spend_per_browser_diff_control,prolist_total_spend_per_browser_diff_lift,prolist_total_spend_per_browser_diff_sig,prolist_total_spend_per_browser_diff_pwr,offsite_ads_revenue_sum_diff_treatment,offsite_ads_revenue_sum_diff_control,offsite_ads_revenue_sum_diff_lift,offsite_ads_revenue_sum_diff_sig,offsite_ads_revenue_sum_diff_pwr
1,cnc.desktop_lp_add_shop_image,on,0.05897,0.05907,-0.00171,0.38982,0.99960,0.10457,0.10495,-0.00360,...,0.07868,0.07817,0.00655,0.00014,0.99999,0.23032,0.23524,-0.02092,0.00000,1.00000
2,cnc.mweb_lp_add_shop_image,on,0.01737,0.01746,-0.00523,0.05969,0.97349,0.05232,0.05246,-0.00267,...,0.06425,0.06419,0.00089,0.53147,1.00000,0.09948,0.09153,0.08686,0.00000,1.00000
4,cnc.free_shipping_badge_ext_lp,on,0.02269,0.02263,0.00236,0.54390,0.82240,0.10047,0.10034,0.00128,...,0.07410,0.07472,-0.00825,0.00007,0.99912,0.11623,0.11474,0.01300,0.00000,1.00000
7,ranking/cnc.update_same_shop_recs_ranker_elp,on,0.02546,0.02529,0.00680,0.03266,0.93402,0.05691,0.05662,0.00520,...,0.07103,0.07196,-0.01292,0.00000,0.99991,0.10874,0.11412,-0.04714,0.00000,1.00000
8,cnc.desktop_lp_add_shop_image_v2,on,0.05448,0.05423,0.00465,0.09359,0.97526,0.09826,0.09853,-0.00273,...,0.09016,0.08962,0.00592,0.00518,0.99899,0.22786,0.21771,0.04662,0.00000,1.00000


In [205]:
results_w_labels = results.copy().reset_index(drop="True")

results_w_labels['cvr_result'] = np.nan
results_w_labels['total_rev_result'] = np.nan
results_w_labels['total_rev_ex_ncp_result'] = np.nan

for index, row in results_w_labels.iterrows():
    
    if (row['cvr_sig']  <= .10) & (row['cvr_pwr'] >= .7):
        
        if row['cvr_lift'] < 0:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('cvr_result')] = 'stat sig negative'
        
        else:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('cvr_result')] = 'stat sig positive'
        
    else:
        
        results_w_labels.iloc[index,results_w_labels.columns.tolist().index('cvr_result')] = 'neutral or not powered'
            
    
    if (row['total_rev_sig']  <= .10) & (row['total_rev_pwr'] >= .7):
        
        if row['total_rev_lift'] < 0:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_result')] = 'stat sig negative'
        
        else:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_result')] = 'stat sig positive'
        
    else:
        
         results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_result')] = 'neutral or not powered'
            
    
    if (row['total_rev_ex_ncp_sig']  <= .10) & (row['total_rev_ex_ncp_pwr'] >= .7):
        
        if row['total_rev_ex_ncp_lift'] < 0:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_ex_ncp_result')] = 'stat sig negative'
        
        else:
            
            results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_ex_ncp_result')] = 'stat sig positive'
        
    else:
        
         results_w_labels.iloc[index,results_w_labels.columns.tolist().index('total_rev_ex_ncp_result')] = 'neutral or not powered'
        
            

In [206]:
results_w_labels.head()

,experiment_id,variant_id,cvr_treatment,cvr_control,cvr_lift,cvr_sig,cvr_pwr,prolist_rev_treatment,prolist_rev_control,prolist_rev_lift,...,prolist_total_spend_per_browser_diff_sig,prolist_total_spend_per_browser_diff_pwr,offsite_ads_revenue_sum_diff_treatment,offsite_ads_revenue_sum_diff_control,offsite_ads_revenue_sum_diff_lift,offsite_ads_revenue_sum_diff_sig,offsite_ads_revenue_sum_diff_pwr,cvr_result,total_rev_result,total_rev_ex_ncp_result
0,cnc.desktop_lp_add_shop_image,on,0.05897,0.05907,-0.00171,0.38982,0.99960,0.10457,0.10495,-0.00360,...,0.00014,0.99999,0.23032,0.23524,-0.02092,0.00000,1.00000,neutral or not powered,neutral or not powered,neutral or not powered
1,cnc.mweb_lp_add_shop_image,on,0.01737,0.01746,-0.00523,0.05969,0.97349,0.05232,0.05246,-0.00267,...,0.53147,1.00000,0.09948,0.09153,0.08686,0.00000,1.00000,stat sig negative,neutral or not powered,neutral or not powered
2,cnc.free_shipping_badge_ext_lp,on,0.02269,0.02263,0.00236,0.54390,0.82240,0.10047,0.10034,0.00128,...,0.00007,0.99912,0.11623,0.11474,0.01300,0.00000,1.00000,neutral or not powered,neutral or not powered,neutral or not powered
3,ranking/cnc.update_same_shop_recs_ranker_elp,on,0.02546,0.02529,0.00680,0.03266,0.93402,0.05691,0.05662,0.00520,...,0.00000,0.99991,0.10874,0.11412,-0.04714,0.00000,1.00000,stat sig positive,neutral or not powered,neutral or not powered
4,cnc.desktop_lp_add_shop_image_v2,on,0.05448,0.05423,0.00465,0.09359,0.97526,0.09826,0.09853,-0.00273,...,0.00518,0.99899,0.22786,0.21771,0.04662,0.00000,1.00000,stat sig positive,neutral or not powered,neutral or not powered


In [207]:
results_w_labels.to_csv('lex_exp.csv', index=False)